Download dataset from kaggle


In [1]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/l33tc0d3r/indian-food-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: sherifmagdyabdellah
Your Kaggle Key: ··········


100%|██████████| 1.48G/1.48G [00:10<00:00, 156MB/s]


store all images in one folder

In [3]:
import os
import shutil
import numpy as np
from sklearn.preprocessing import LabelEncoder
import cv2
import tensorflow as tf
from sklearn.utils import shuffle

!mkdir all_images

train_dir = '/content/indian-food-classification'
dest_dir = '/content/all_images'
counter = 0
labels=[]
files_path=[]
for subdir, dirs, files in os.walk(train_dir):
    for file in files:
      if(file.split(".")[-1] in ["png","jpg","jpeg"]):
          full_path = os.path.join(subdir, file)
          full_path_dest = os.path.join(dest_dir,f"={subdir.split('/')[-1]}={file}")
          img = cv2.imread(full_path)
          img=cv2.resize(img,(240,240))
          cv2.imwrite(full_path_dest, img)
          labels.append(subdir.split('/')[-1])
          files_path.append(full_path_dest)
          counter = counter + 1
print(counter)

6256


In [4]:
le=LabelEncoder()
le.fit(labels)
labels_en = le.transform(labels)

In [5]:
print(f"ours labels encode{np.unique(labels_en)}")
print("we have 20 differnet class")

ours labels encode[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
we have 20 differnet class


save our data pathes and labels

In [6]:
np.save('files_path.npy', files_path)
np.save('labels_en.npy', labels_en)

In [7]:

files_path_shuffle,labels_en_shuffle=shuffle(files_path, labels_en)

In [8]:
np.save('files_path_shuffle.npy', files_path_shuffle)
np.save('labels_en_shuffle.npy', labels_en_shuffle)

create  our Custom Generator

In [9]:


class My_Custom_Generator(tf.keras.utils.Sequence) :
  
  def __init__(self, image_filenames, labels, batch_size) :
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int32)
  
  
  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    return (np.array([cv2.imread(str(file_name))for file_name in batch_x])/255
            , np.array(batch_y))

In [10]:
from sklearn.model_selection import train_test_split
X_train_filenames, X_val_filenames, y_train, y_val = train_test_split(files_path_shuffle, labels_en_shuffle, test_size=0.2, random_state=1)

In [11]:
X_train_filenames=np.array(X_train_filenames)
X_val_filenames=np.array(X_val_filenames)
y_train=np.array(y_train)
y_val=np.array(y_val)

In [12]:
print(X_train_filenames.shape)
print(y_train.shape)

print(X_val_filenames.shape)
print(y_val.shape)

np.save('X_train_filenames.npy', X_train_filenames)
np.save('y_train.npy', y_train)

np.save('X_val_filenames.npy', X_val_filenames)
np.save('y_val.npy', y_val)

(5004,)
(5004,)
(1252,)
(1252,)


create our model

In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

VGG16=tf.keras.applications.VGG16(
    include_top=False,
    input_shape=(240,240,3))

for layer in VGG16.layers[:]:
    layer.trainable = False


In [19]:
model = keras.Sequential()
model.add(VGG16)
model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(20, activation='softmax'))
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 100)               2508900   
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 20)                2020      
                                                                 
Total params: 17,225,608
Trainable params: 2,510,920
Non-trainable params: 14,714,688
_________________________________________________________________


In [20]:
batch_size = 64

my_training_batch_generator = My_Custom_Generator(X_train_filenames, y_train, batch_size)
my_validation_batch_generator = My_Custom_Generator(X_val_filenames, y_val, batch_size)

In [21]:
model.fit_generator(generator=my_training_batch_generator,
                   steps_per_epoch =len(X_train_filenames) // batch_size,
                   epochs = 5,
                   verbose = 1,
                   validation_data = my_validation_batch_generator,
                   validation_steps = len(X_val_filenames) // batch_size)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


78/78 [==============================] - 46s 516ms/step - loss: 2.5144 - accuracy: 0.2767 - val_loss: 1.7481 - val_accuracy: 0.5066
Epoch 2/5
78/78 [==============================] - 38s 484ms/step - loss: 1.4776 - accuracy: 0.5593 - val_loss: 1.4076 - val_accuracy: 0.6003
Epoch 3/5
78/78 [==============================] - 38s 481ms/step - loss: 1.0709 - accuracy: 0.6806 - val_loss: 1.2701 - val_accuracy: 0.6332
Epoch 4/5
78/78 [==============================] - 37s 479ms/step - loss: 0.8197 - accuracy: 0.7609 - val_loss: 1.1981 - val_accuracy: 0.6562
Epoch 5/5
78/78 [==============================] - 38s 482ms/step - loss: 0.6471 - accuracy: 0.8219 - val_loss: 1.1814 - val_accuracy: 0.6513


here i don't care about model or to score highest accuracy but our goal to train model with large dataset which doesn't fit in memory